|**Name**| **ID #** | **Email** | **When Midterm**| 
| :--     |  :--      |  :--    | :--             |           
|Xuan Xu |   111675940 | xuaxu@cs.stonybrook.edu        |                 |             
|Hae-Na Lee|          |         |                 |            
|Caitao Zhan| 111634527| caitao.zhan@stonybrook.edu | 10/16 (CSE548) |



In [ ]:
print("Hello Homework3!")
print("Always remember to git pull before you git push.")
print("When a conflict occurs, don't be nervous. Just manually fix the conflicts.")
print("Also, you can .gitignore whatever you don't want others see.")
print("Hope we can finish task 1,2,3 by Sunday night (10/8).")

In [ ]:
# import all packages needed

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, time
from sklearn import linear_model

In [ ]:
# To all: let's all use the following names:
# 'df_house16', 'df_house17', 'df_train16', 'df_train17', 'df_merge16', 'df_merge17'

df_house16 = pd.read_csv('../../data/properties_2016.csv')
df_train16 = pd.read_csv('../../data/train_2016_v2.csv')
df_merge16 = pd.merge(df_house16, df_train16, on='parcelid')

#df_house17 = pd.read_csv('../../data/properties_2017.csv')
#df_train17 = pd.read_csv('../../data/train_2017.csv')
#df_merge17 = pd.merge(df_house17, df_train17, on='parcelid')

# Task 1

# Task 2

# Task 3